#  <center> Problem Set 3 <center>

<center> 3.C01/3.C51, 7.C01/7.C51, 10.C01/10.C51, 20.C01/20.C51 <center>

<b>Name:</b>

<b>Kerberos id:</b>

In [ ]:
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib
import torch
from torch import nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

matplotlib.rcParams.update({'font.size': 15})
matplotlib.rc('lines', linewidth=3, color='g')
matplotlib.rcParams['axes.linewidth'] = 2.0
matplotlib.rcParams['axes.linewidth'] = 2.0
matplotlib.rcParams["xtick.major.size"] = 6
matplotlib.rcParams["ytick.major.size"] = 6
matplotlib.rcParams["ytick.major.width"] = 2
matplotlib.rcParams["xtick.major.width"] = 2
matplotlib.rcParams['text.usetex'] = False

## <center>Problem 1: Predicting DNA Binding Sites</center>

In [ ]:
# Download data 
! wget https://raw.githubusercontent.com/vikram-sundar/ML4MolEng_Spring2022/master/psets/ps3/data/train_dna.csv

In [ ]:
train_pd = pd.read_csv("./train_dna.csv")

X = train_pd.seq.values
y = train_pd.bind.values

### 1.1 (15 points) Build Datasets and DataLoaders in PyTorch

One-hot encode DNA sequence data.

In [ ]:
def SeqEnc(sequences):
    # Write your one-hot encoding code here
    
    # sequences - an array of DNA sequences as strings
    
X = SeqEnc(X)

Implement your dataset class that takes in your data.

In [ ]:
# Generate dataset 
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.Tensor(X)  # store X as a pytorch Tensor
        self.y = torch.Tensor(y)  # store y as a pytorch Tensor
        self.len=len(self.X)      # number of samples in the data 

    def __getitem__(self, index):
        # your implementation here: 

    def __len__(self):
        return self.len
    

Define your Datasets and DataLoaders.

In [ ]:
# Define dataset 
X_train = # fill in code here
y_train = # fill in code here
X_val = # fill in code here
y_val = # fill in code here
X_test = # fill in code here
y_test = # fill in code here

#Build Dataset 
train_data = # fill in code here
val_data = # fill in code here
test_data = # fill in code here

# Build DataLoader 
batch_size = 256
train_loader = # fill in code here
val_loader = # fill in code here
test_loader = # fill in code here

What is the benefit of batching your data into mini-batches versus using the entire dataset to optimize the model all at once?

### 1.2 (20 points) Build an LSTM-based binding classifier 

The following example will help familiarize you with the LSTM.

In [ ]:
# Define a LSTM module 
lstm_model = nn.LSTM(input_size=4, hidden_size=16, batch_first=True).to("cuda:0") # "cuda:0" is the device id

# Send your batch to a GPU 
X_batch, y_batch = next(iter(train_loader))
X_batch = X_batch.to("cuda:0")
y_batch = y_batch.to("cuda:0")

# Propagate your batch into your model 
lstm_out, (ht, ct) = lstm_model(X_batch) 
print(lstm_out.shape, ht.shape, ct.shape)

# You can play with hyperparameters to see how your output changes

Now build your LSTM-based classifier as a nn.Module.

In [ ]:
class LSTMseq(torch.nn.Module) :
    def __init__(self, input_dim, hidden_dim) :
        super().__init__()
        
        # Define a LSTM module
        
        # Define a MLP regressor 
        
        # Define a sigmoid transform
        
    def forward(self, x):
        
        # Apply LSTM 
        
        # Pass output into a MLP 
        
        # Transform output into probabilites 
        
        # Return probabilities 
        
        return proba

In [ ]:
# Test your output on a batch 
clf = LSTMseq(input_dim=4, hidden_dim=16).to('cuda:0')
print(clf(X_batch).shape)

### 1.3 (25 points) Implement functions for training and testing

Your optimizer and scheduler.

In [ ]:
device = 'cuda:0'
model = LSTMseq(4, 16).to(device)

optimizer = # complete optimizer here
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.5)

The training and validation loops and evaluation function.

In [ ]:
def train(model, dataloader, optimizer, device):
    
    '''
    A function to train on the entire dataset for one epoch.
    
    Args: 
        model (torch.nn.Module): Your sequence classifier 
        dataloader (torch.utils.data.Dataloader): DataLoader object for the train data
        optimizer (torch.optim.Optimizer): Optimizer object to interface gradient calculation and optimization 
        device (str): Your device
        
    Returns: 
        float: loss averaged over all the batches 
    
    '''

    batch_loss = []
    model.train() # Set model to training mode 
    
    for batch in dataloader:    
        seq, label = batch
        seq = seq.to(device)
        label = label.to(device)
        
        # train your model on each batch here 

    return 0.0


def validate(model, dataloader, device):
    
    '''
    A function to validate on the validation dataset for one epoch.
    
    Args: 
        model (torch.nn.Module): Your sequence classifier 
        dataloader (torch.utils.data.Dataloader): DataLoader object for the validation data
        device (str): Your device
        
    Returns: 
        float: loss averaged over all the batches 
    
    '''
    
    val_loss = []
    model.eval() # Set model to evaluation mode 
    with torch.no_grad():    
        for batch in dataloader:
            seq, label = batch
            seq = seq.to(device)
            label = label.to(device)
            
            # validate your model on each batch here 
            
    return 0.0    

def evaluate(model, dataloader, device):

    '''
    A function to return the classification probabilities and true labels (for evaluation). 
    
    Args: 
        model (torch.nn.Module): your sequence classifier 
        dataloader (torch.utils.data.Dataloader): DataLoader object for the train data
        device (str): Your device
        
    Returns: 
        (np.array, np.array): true labels, predicted probabilities
    '''

    pred_prob = []
    labels = []
    with torch.no_grad():
        model.eval()
        for batch in dataloader:
            epoch_loss = []
            seq, label = batch

            seq = seq.to(device)
            label = label.to(device)
            
            # evaluate your model here
            
    return labels, pred_prob

In [ ]:
print("epoch", "train loss", "validation loss")

val_loss_curve = []
train_loss_curve = []

for epoch in range(500):
    
    # Compute train your model on training data
    epoch_loss = train(model, train_loader, optimizer,  device=0)
    
    # Validate your on validation data 
    val_loss = validate(model, val_loader, device=0) 
    
    # Record train and loss performance 
    train_loss_curve.append(epoch_loss)
    val_loss_curve.append(val_loss)
    
    # The learning rate scheduler records the validation loss 
    scheduler.step(val_loss)
    
    print(epoch, epoch_loss, val_loss)

Plot train and validation loss functions.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(val_loss_curve, label='Validation Loss')
ax.plot(train_loss_curve, label='Training Loss')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.legend(loc='upper right')
fig.tight_layout()

Report the AUC on test data.

In [ ]:
# Code to compute AUC on test data 

    
print("AUC on the test dataset is {}.".format(test_score) ) 

## Part 2: Dimensionality Reduction for Molecular Representations

In [ ]:
! wget https://raw.githubusercontent.com/vikram-sundar/ML4MolEng_Spring2022/master/psets/ps3/data/drug.csv
! wget https://raw.githubusercontent.com/vikram-sundar/ML4MolEng_Spring2022/master/psets/ps3/data/morgan.csv

In [ ]:
# load data 
df = pd.read_csv("drug.csv")
fp = np.loadtxt('morgan.csv', delimiter=',')

assert fp.shape[0] == df.shape[0]

### 2.1 (10 points) Principal Component Analysis on Molecular Fingerprints

Perform PCA to reduce data into vectors of 100 dimensions.

In [ ]:

# skeleton code for plotting

fig, ax = plt.subplots(figsize=(5,5))
ax.scatter([0], [0], s=3, label='inactive') 
ax.scatter([0], [0], color='red', s= 3, label='active')
ax.legend()

What is the explained variance ratio of the 100 principal components?

In [ ]:


print("The first 100 components explains {} of the total variance".format(1.0) )

What patterns do you observe?

### 2.2 (10 points) t-SNE analysis on Molecular Fingerprints

Perform t-SNE on the obtained principal components, with perplexity value of 2, 30, and 500. Plot the results and label your plots.

What differences do you see between the 3 t-SNE plots? What patterns do you observe in the perplexity = 30 plot?

### 2.3 (15 points) Are the low dimensional embeddings meaningful?

Split the data into 10 folds. For each fold, train on the other 9 folds and validate on the last fold. Record your prediction.

Classify your predictions into True Positives (TP), True Negatives (TN), False Positives (FP) and False Negatives (FN).

Plot the 2D t-SNE embeddings (perplexity = 30) colored by the four classification classes.

What pattern do you observe? 